In [2]:
import pandas as pd
import numpy as np
import os

In [70]:
# path
path = r"C:\Users\USYS671257\OneDrive - WSP O365\21_31000110.002_Chattanooga TPO Model\model outputs"

# inputs
tourdata  = pd.read_csv(os.path.join(path, "_tour_2.dat"), sep = '\t')
perdata = pd.read_csv(os.path.join(path, "_person_2.dat"), sep = '\t')
hhdata = pd.read_csv(os.path.join(path, "_household_2.dat"), sep = '\t')

# parameters
excludeChildren5 = True

In [71]:
def prep_perdata(perdata,hhdata):
    
    hhdata["vehcat"] = np.where(hhdata.hhvehs>0, ">0-Veh HHs", "0-Veh HHs")
    hhdata = hhdata[["hhno","vehcat"]]
    perdata = pd.merge(perdata, hhdata, on="hhno",how="left")
    
    return perdata



def prep_tomode(tourdata):

    tourdata['tourmode'] = 'no mode'
    tourdata.loc[tourdata['tmodetp'] == 1, 'tourmode'] = 'Walk'
    tourdata.loc[tourdata['tmodetp'] == 2, 'tourmode'] = 'Bike'
    tourdata.loc[tourdata['tmodetp'] == 3, 'tourmode'] = 'Drive Alone'
    tourdata.loc[tourdata['tmodetp'] == 4, 'tourmode'] = 'Shared Ride 2'
    tourdata.loc[tourdata['tmodetp'] == 5, 'tourmode'] = 'Shared Ride 3+'
    tourdata.loc[tourdata['tmodetp'] == 6, 'tourmode'] = 'Walk-Transit'
    tourdata.loc[tourdata['tmodetp'] == 7, 'tourmode'] = 'Drive-Transit'
    tourdata.loc[tourdata['tmodetp'] == 8, 'tourmode'] = 'School Bus'

    return tourdata



def prep_tourdata(tourdata, perdata, hhdata):

    """ prepare tourdata """
    
    perdata = prep_perdata(perdata,hhdata)
    perdata = perdata[["hhno","pno","pptyp","vehcat","psexpfac"]]
    tourdata = prep_tomode(tourdata)
    
    tourdata = pd.merge(tourdata, perdata, on =["hhno","pno"], how="left")
    if excludeChildren5:
        tourdata = tourdata[tourdata["pptyp"]<8]
    tourdata["pdpurp"] = np.where(tourdata.pdpurp==8, 7, tourdata.pdpurp)   # combine recreational 8 with socail 7
    tourdata["pdpurp"] = np.where(tourdata.pdpurp==9, 4, tourdata.pdpurp)   # combine medical 8 with personal business 4
    tourdata["pdpurp2"] = np.where(tourdata.parent==0, tourdata.pdpurp, 8)   # workbased trips

    wrktours = tourdata[tourdata["pdpurp"]==1]
    wrktours = wrktours[["hhno","pno","tour","tourmode"]]
    wrktours = wrktours.rename(columns={"tour":"parent", "tourmode":"parenttourmode"})
    wrkbasedtours = tourdata[tourdata["parent"]>0]
    wrkbasedtours = pd.merge(wrkbasedtours,wrktours,on=["hhno","pno","parent"],how="left")

    nonwrkbasedtours = tourdata[tourdata["parent"]==0]
    nonwrkbasedtours.loc[:,"parenttourmode"]=0

    wrkbasedtours = wrkbasedtours[nonwrkbasedtours.columns]
    tourdata = pd.concat([nonwrkbasedtours,wrkbasedtours])

    return(tourdata)



def summary_tour_mode(dstourdata, purpose):
    
    """ summarize tour mode by HHvehicles and destination purpose 
        Work 1 
        School 2 
        Escort 3
        Personal_Business 4
        Shop 5
        Meal 6
        SocRec 7
        Workbased 8    
    """
    
    dstourdata = dstourdata[dstourdata["pdpurp2"]==purpose]
    
    
    column_order = ["0-Veh HHs", ">0-Veh HHs"]
    index_order = ['Drive Alone', 'Shared Ride 2', 'Shared Ride 3+', 'Drive-Transit', 'Walk-Transit',
                    'Bike','Walk','School Bus']
    
    summary = (dstourdata.groupby(["tourmode", "vehcat"])["psexpfac"].
                          sum().
                          reset_index().
                          pivot_table(values='psexpfac', 
                                     index="tourmode",
                                     columns="vehcat",
                                     fill_value=0).
                           reindex(columns=column_order,
                                   index=index_order).
                           fillna(0))
    
    return summary

In [72]:
# clean up tour data
dstourdata = prep_tourdata(tourdata, perdata, hhdata)

In [73]:
# tour mode - Work 1 
summary_tour_mode(dstourdata, purpose=1)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,132565.0
Shared Ride 2,292.0,23496.0
Shared Ride 3+,215.0,13955.0
Drive-Transit,20.0,216.0
Walk-Transit,81.0,89.0
Bike,80.0,79.0
Walk,882.0,1044.0
School Bus,0.0,0.0


In [74]:
# tour mode - School 2 
summary_tour_mode(dstourdata, purpose=2)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,5319.0
Shared Ride 2,271.0,15176.0
Shared Ride 3+,510.0,35808.0
Drive-Transit,0.0,0.0
Walk-Transit,185.0,119.0
Bike,21.0,40.0
Walk,220.0,1010.0
School Bus,1516.0,15636.0


In [75]:
# tour mode - Escort 3
summary_tour_mode(dstourdata, purpose=3)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,88.0
Shared Ride 2,1948.0,32481.0
Shared Ride 3+,1719.0,31094.0
Drive-Transit,0.0,0.0
Walk-Transit,0.0,0.0
Bike,244.0,14.0
Walk,552.0,48.0
School Bus,0.0,0.0


In [76]:
# tour mode - Personal_Business 4
summary_tour_mode(dstourdata, purpose=4)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,37236.0
Shared Ride 2,3390.0,20412.0
Shared Ride 3+,1866.0,21166.0
Drive-Transit,0.0,0.0
Walk-Transit,695.0,25.0
Bike,10.0,3.0
Walk,431.0,398.0
School Bus,0.0,0.0


In [77]:
# tour mode - Shop 5
summary_tour_mode(dstourdata, purpose=5)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,23256.0
Shared Ride 2,2083.0,12252.0
Shared Ride 3+,1201.0,12171.0
Drive-Transit,0.0,0.0
Walk-Transit,246.0,6.0
Bike,4.0,5.0
Walk,294.0,237.0
School Bus,0.0,0.0


In [78]:
# tourmode - Meal 6
summary_tour_mode(dstourdata, purpose=6)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,5847.0
Shared Ride 2,1036.0,12334.0
Shared Ride 3+,595.0,10227.0
Drive-Transit,0.0,0.0
Walk-Transit,103.0,3.0
Bike,0.0,0.0
Walk,181.0,366.0
School Bus,0.0,0.0


In [79]:
# tour mode - SocRec 7
summary_tour_mode(dstourdata, purpose=7)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,18132.0
Shared Ride 2,1581.0,15278.0
Shared Ride 3+,929.0,16693.0
Drive-Transit,0.0,0.0
Walk-Transit,226.0,11.0
Bike,5.0,8.0
Walk,323.0,452.0
School Bus,0.0,0.0


In [80]:
# tour mode - Workbased 8
summary_tour_mode(dstourdata, purpose=8)

vehcat,0-Veh HHs,>0-Veh HHs
tourmode,,
Drive Alone,0.0,15531.0
Shared Ride 2,45.0,4106.0
Shared Ride 3+,26.0,3116.0
Drive-Transit,0.0,0.0
Walk-Transit,0.0,33.0
Bike,1.0,38.0
Walk,85.0,2961.0
School Bus,0.0,0.0
